In [ ]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.model_selection import train_test_split
import os
import cv2 as cv
import numpy as np
import mediapipe as mp
import time
import matplotlib.pyplot as plt

In [ ]:
filenames = os.listdir('/home/dsxuser/work/train_Data/train')
categories = []
for filename in filenames:
    category = filename.split(' ')[0]
    if category == 'Iris Julian':
        categories.append('Iris Julian')
    elif category == 'Iris Oriana':
        categories.append('Iris Oriana')
    elif category == 'Iris Marcelo':
        categories.append('Iris Marcelo')
    else :
        categories.append('Iris Desconocido')
        
df = pd.DataFrame({
    'filename': filenames,
    'category': categories
})

df = df.sample(frac=1).reset_index(drop=True)

In [ ]:
len(filenames)

In [ ]:
df.head(10)

In [ ]:
df.tail(10)

In [ ]:
df['category'].value_counts().plot.bar()

In [ ]:
sample = random.choice(filenames)
image = load_img("/home/dsxuser/work/train_Data/train/"+sample)
plt.imshow(image)

In [ ]:
train_df, validate_df = train_test_split(df, test_size=0.30, random_state=42)
train_df = train_df.reset_index(drop=True)
validate_df = validate_df.reset_index(drop=True)

In [ ]:
train_df['category'].value_counts().plot.bar()

In [ ]:
validate_df['category'].value_counts().plot.bar()

In [ ]:
total_train = train_df.shape[0]
total_validate = validate_df.shape[0]

In [ ]:
# Load the pretrained ResNet50 model
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False  # Freeze the base model

# Add custom layers on top of ResNet50
x = base_model.output
x = GlobalAveragePooling2D()(x)  # Reduce dimensionality and prevent overfitting
x = Dense(1024, activation='relu')(x)
x = BatchNormalization()(x)  # Helps to normalize the inputs of the activation layers
x = Dropout(0.5)(x)  # Randomly sets input units to 0 at each step during training, which helps prevent overfitting
x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.25)(x)
predictions = Dense(4, activation='softmax')(x)  # Final prediction layer with softmax for 4 classes

# Finalizing the model
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Model summary
model.summary()

In [2]:
#Early Stop
#To prevent over fitting we will stop the learning 
#after 10 epochs and if val_loss value has not decreased.
earlystop = EarlyStopping(patience=10)

#Learning Rate Reduction
#We will reduce the learning rate when then accuracy does not increase for 2 steps
learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', 
                                            patience=2, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.00001)

callbacks = [earlystop, learning_rate_reduction]

NameError: name 'EarlyStopping' is not defined

In [ ]:
# Set up the image data generators for training and validation sets
train_dir = './iris_data'  # CHANGE path to the training data directory
batch_size = 32
image_size = (224, 224)

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=0,
    width_shift_range=0,
    height_shift_range=0,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.5
)

train_generator = train_datagen.flow_from_dataframe(
    train_df,
    train_dir,
    x_col='filename',
    y_col='category',
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

validation_generator = train_datagen.flow_from_dataframe(
    validate_df,
    train_dir,
    x_col='filename',
    y_col='category',
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

# Train the model
epochs = 50  # Number of epochs to train for
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size,
    epochs=epochs
    callbacks=callbacks
)

In [ ]:
Validation_Accuracy = model.evaluate_generator(generator=validation_generator, steps=20)

In [ ]:
print("The Model Accuracy for Validation data is: ", "{:.2%}".format(Validation_Accuracy[1]))

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 12))
ax1.plot(history.history['loss'], color='b', label="Training loss")
ax1.plot(history.history['val_loss'], color='r', label="validation loss")
ax1.set_xticks(np.arange(1, epochs, 1))
ax1.set_yticks(np.arange(0, 1, 0.1))

ax2.plot(history.history['acc'], color='b', label="Training accuracy")
ax2.plot(history.history['val_acc'], color='r',label="Validation accuracy")
ax2.set_xticks(np.arange(1, epochs, 1))

legend = plt.legend(loc='best', shadow=True)
plt.tight_layout()
plt.show()

In [ ]:
mp_face_mesh = mp.solutions.face_mesh

LEFT_IRIS = [474, 475, 476, 477]
RIGHT_IRIS = [469, 470, 471, 472]

# Función principal
def capture_iris_image():
    cap = cv.VideoCapture(0)
    captured_image = None
    eye_detected_time = None
    frame_with_contours = None

    with mp_face_mesh.FaceMesh(
        max_num_faces=1,
        refine_landmarks=True,
        min_detection_confidence=0.5,
        min_tracking_confidence=0.5
    ) as face_mesh:
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            
            frame = cv.flip(frame, 1)
            rgb_frame = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
            img_h, img_w = frame.shape[:2]
            results = face_mesh.process(rgb_frame)
            
            if results.multi_face_landmarks:
                mesh_points = np.array([np.multiply([p.x, p.y], [img_w, img_h]).astype(int) for p in results.multi_face_landmarks[0].landmark])
                
                (l_cx, l_cy), l_radius = cv.minEnclosingCircle(mesh_points[LEFT_IRIS])
                center_left = np.array([l_cx, l_cy], dtype=np.int32)
                (r_cx, r_cy), r_radius = cv.minEnclosingCircle(mesh_points[RIGHT_IRIS])
                center_right = np.array([r_cx, r_cy], dtype=np.int32)
                
                frame_with_contours = frame.copy()
                cv.circle(frame_with_contours, center_left, int(l_radius), (255, 0, 255), 1, cv.LINE_AA)
                cv.circle(frame_with_contours, center_right, int(r_radius), (255, 0, 255), 1, cv.LINE_AA)

                if eye_detected_time is None:
                    eye_detected_time = time.time()  # Inicia el temporizador
                elif time.time() - eye_detected_time >= 5:
                    captured_image = frame.copy()  # Captura la imagen sin contornos
                    break
            else:
                eye_detected_time = None  # Reinicia el temporizador si no se detectan ojos
            
            # Muestra el frame con contornos si están disponibles, de lo contrario muestra el frame original
            if frame_with_contours is not None:
                cv.imshow('img', frame_with_contours)
            else:
                cv.imshow('img', frame)
            
            key = cv.waitKey(1)
            if key == ord('q'):
                break

    cap.release()
    cv.destroyAllWindows()
    return captured_image

# Captura la imagen del iris
captured_image = capture_iris_image()

if captured_image is not None:
    # Convierte la imagen capturada de BGR a RGB
    captured_image_rgb = cv.cvtColor(captured_image, cv.COLOR_BGR2RGB)
    
    # Guarda la imagen como archivo
    cv.imwrite('captured_iris_image.jpg', captured_image_rgb)
    print("Imagen capturada y guardada exitosamente.")
    
    # Plotea la imagen usando matplotlib
    plt.imshow(captured_image_rgb)
    plt.axis('off')  # Oculta los ejes
    plt.show()
else:
    print("No se capturó ninguna imagen.")


In [ ]:
# Load and preprocess an image

img = image.load_img(captured_image, target_size=image_size)
img_array = image.img_to_array(img)
img_array_expanded_dims = np.expand_dims(img_array, axis=0)
img_preprocessed = img_array_expanded_dims / 255.0

# Predict the class
prediction = model.predict(img_preprocessed)
predicted_class = np.argmax(prediction, axis=1)

print("Predicted class:", predicted_class)